In [ ]:
# Notebook 01: Data setup for BulkRNABert repro
print('Notebook loaded correctly!')

In [16]:
# --- Repo + path setup ---

import os, sys

# 1) Clone repo if it doesn't exist
if not os.path.exists('/content/bdh_challenge_2025'):
    %cd /content
    !git clone https://github.com/arionandrei2000/bdh_challenge_2025.git

# 2) Move into repo root
%cd /content/bdh_challenge_2025
print("PWD:", os.getcwd())
print("Repo contents:", os.listdir())

# 3) src is under notebooks/src, so add that parent folder to sys.path
src_parent = "/content/bdh_challenge_2025/notebooks"
if src_parent not in sys.path:
    sys.path.append(src_parent)

print("sys.path updated; ready to import src.")


/content/bdh_challenge_2025
PWD: /content/bdh_challenge_2025
Repo contents: ['.git', 'notebooks', 'README.md']
sys.path updated; ready to import src.


In [18]:

import numpy as np
import pandas as pd

from src.preprocess import log_normalize, max_normalize, bin_expression

